### **2 - Atributos**

In [1]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')

### **Ir hacia abajo**

Las etiquetas pueden contener strings y otras etiquetas. Estos elementos son los hijos de la etiqueta. Beautiful Soup proporciona muchos atributos diferentes para navegar e iterar sobre los hijos de una etiqueta.

Ten en cuenta que los strings de Beautiful Soup no soportan ninguno de estos atributos, porque un string no puede tener hijos.

#### **`.contents`**

In [5]:
head_tag = soup.head
head_tag

<head><title>The Dormouse's story</title></head>

In [6]:
head_tag.contents

[<title>The Dormouse's story</title>]

In [7]:
title_tag = head_tag.contents[0]
title_tag

<title>The Dormouse's story</title>

In [8]:
title_tag.contents

["The Dormouse's story"]

#### **`.children`**

El propio objeto `BeautifulSoup` tiene hijos (children). En este caso, la etiqueta `<html>` es hija del objeto `BeautifulSoup`:

In [9]:
len(soup.contents)

2

In [10]:
soup.contents

['\n',
 <html><head><title>The Dormouse's story</title></head>
 <body>
 <p class="title"><b>The Dormouse's story</b></p>
 <p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at the bottom of a well.</p>
 <p class="story">...</p>
 </body></html>]

In [11]:
soup.contents[1]

<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body></html>

In [12]:
soup.contents[1].name

'html'

Un `string` no tiene `.contents`, porque no puede contener nada:

In [13]:
text = title_tag.contents[0]
text.contents

AttributeError: 'NavigableString' object has no attribute 'contents'

En lugar de obtenerlos como una lista, puedes iterar sobre los hijos de una etiqueta (tag's children) utilizando el generador `.children`:

In [ ]:
for child in title_tag.children:
    print(child)

The Dormouse's story


#### **`.descendants`**

Los atributos `.contents` y `.children` sólo tienen en cuenta los hijos directos de una etiqueta. Por ejemplo, la etiqueta `<head>` tiene un único hijo directo: la etiqueta `<title>`:

In [ ]:
head_tag.contents

[<title>The Dormouse's story</title>]

Pero la propia etiqueta `<title>` tiene un hijo: el string "`The Dormouse's story`". En cierto sentido, este string también es hijo de la etiqueta `<head>`. El atributo `.descendants` permite recorrer recursivamente todos los hijos de una etiqueta: sus hijos directos, los hijos de sus hijos directos, etc:

In [ ]:
for child in head_tag.descendants:
    print(child)

<title>The Dormouse's story</title>
The Dormouse's story


La etiqueta `<head>` sólo tiene un hijo, pero tiene dos descendientes: la etiqueta `<title>` y el hijo de la etiqueta `<title>`. El objeto BeautifulSoup sólo tiene un hijo directo (la etiqueta `<html>`), pero tiene un montón de descendientes:

In [ ]:
len(list(soup.children))

2

In [ ]:
len(list(soup.descendants))

27

#### **`.string`**

Si una etiqueta sólo tiene un hijo, y ese hijo es un `NavigableString`, el hijo se pone a disposición como `.string`:

In [14]:
title_tag.string

"The Dormouse's story"

Si el único hijo de una etiqueta es otra etiqueta, y esa etiqueta tiene un `.string`, se considera que la etiqueta padre tiene el mismo `.string` que su hijo:

In [15]:
head_tag.contents

[<title>The Dormouse's story</title>]

In [16]:
head_tag.string

"The Dormouse's story"

Si una etiqueta contiene más de una cosa, no está claro a qué debe referirse `.string`, por lo que `.string` se define como `None`:

In [17]:
print(soup.html.string)

None


#### **`.strings` y `stripped_strings`**

Si hay más de una cosa dentro de una etiqueta, puedes ver sólo los strings. Utilice el generador `.strings`:

In [20]:
for string in soup.strings:
    print(repr(string))

'\n'
"The Dormouse's story"
'\n'
'\n'
"The Dormouse's story"
'\n'
'Once upon a time there were three little sisters; and their names were\n'
'Elsie'
',\n'
'Lacie'
' and\n'
'Tillie'
';\nand they lived at the bottom of a well.'
'\n'
'...'
'\n'


Estos strings tienden a tener muchos espacios en blanco extra, que puedes eliminar usando el generador `.stripped_strings` en su lugar:

In [21]:
for string in soup.stripped_strings:
    print(repr(string))

"The Dormouse's story"
"The Dormouse's story"
'Once upon a time there were three little sisters; and their names were'
'Elsie'
','
'Lacie'
'and'
'Tillie'
';\nand they lived at the bottom of a well.'
'...'


En este caso, se ignoran las cadenas formadas exclusivamente por espacios en blanco, y se eliminan los espacios en blanco al principio y al final de las cadenas.

### **Ir hacia arriba**

Siguiendo con la analogía del "árbol genealógico", cada etiqueta y cada cadena tiene un padre: la etiqueta que la contiene.

#### **`.parent`**

Puede acceder al elemento padre de un elemento con el atributo `.parent`. En el ejemplo del documento "Tres hermanas", la etiqueta `<head>` es el elemento padre de la etiqueta `<title>`:

In [22]:
title_tag = soup.title
title_tag

<title>The Dormouse's story</title>

In [99]:
title_tag.parent

<head><title>The Dormouse's story</title></head>

In [100]:
title_tag.parent.string

"The Dormouse's story"

El propio título tiene un padre: la etiqueta `<title>` que lo contiene:

In [24]:
title_tag.string.parent

<title>The Dormouse's story</title>

El padre de una etiqueta de nivel superior como `<html>` es el propio objeto `BeautifulSoup`:

In [25]:
html_tag = soup.html
type(html_tag.parent)

bs4.BeautifulSoup

Y el `.parent` de un objeto `BeautifulSoup` se define como None:

In [26]:
print(soup.parent)

None


#### **`.parents`**

Puede iterar sobre todos los padres de un elemento con `.parents`. Este ejemplo utiliza `.parents` para viajar desde una etiqueta `<a>` enterrada en lo más profundo del documento hasta la parte superior del mismo:

In [27]:
link = soup.a
link

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [28]:
for parent in link.parents:
    if parent is None:
        print(parent)
    else:
        print(parent.name)

p
body
html
[document]


### **Ir hacia ambos lados (arriba y abajo)**

Considere un documento sencillo como éste:

In [29]:
sibling_soup = BeautifulSoup("<a><b>text1</b><c>text2</c></b></a>")
print(sibling_soup.prettify())

<html>
 <body>
  <a>
   <b>
    text1
   </b>
   <c>
    text2
   </c>
  </a>
 </body>
</html>



La etiqueta `<b>` y la etiqueta `<c>` están al mismo nivel: ambas son hijas directas de la misma etiqueta. Los llamamos hermanos. Cuando se imprime un documento, los hermanos aparecen en el mismo nivel de sangría. También puedes utilizar esta relación en el código que escribas.

#### **`.next_sibling` y `.previous_sibling`**

Puede utilizar `.next_sibling` y `.previous_sibling` para navegar entre elementos de página que se encuentren en el mismo nivel del árbol de análisis sintáctico:

In [30]:
sibling_soup.b.next_sibling

<c>text2</c>

In [31]:
sibling_soup.c.previous_sibling

<b>text1</b>

La etiqueta `<b>` tiene un `.next_sibling`, pero no un `.previous_sibling`, porque no hay nada antes de la etiqueta `<b>` en el mismo nivel del árbol. Por la misma razón, la etiqueta `<c>` tiene un `.previous_sibling` pero no  un `.next_sibling`:

In [32]:
print(sibling_soup.b.previous_sibling)
print(sibling_soup.c.next_sibling)

None
None


Los strings "`text1`" y "`text2`" no son hermanos, porque no tienen el mismo padre:

In [33]:
sibling_soup.b.string

'text1'

In [36]:
print(sibling_soup.b.string.next_sibling)

None


En los documentos reales, el `.next_sibling` o `.previous_sibling` de una etiqueta suele ser un string que contiene espacios en blanco. Volviendo al documento de las "tres hermanas":
```
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a>
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>
```

Se podría pensar que el `.next_sibling` de la primera etiqueta `<a>` sería la segunda etiqueta `<a>`. Pero en realidad es un string: la coma y la nueva línea que separan la primera etiqueta `<a>` de la segunda:

In [37]:
link = soup.a
link

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [38]:
link.next_sibling

',\n'

La segunda etiqueta `<a>` es en realidad el .next_sibling de la coma:

In [39]:
link.next_sibling.next_sibling

<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>

#### **`.next_siblings` and `.previous_siblings`**

Puede iterar sobre los hermanos de una etiqueta con `.next_siblings` o `.previous_siblings`:

In [40]:
for sibling in soup.a.next_siblings:
    print(repr(sibling))

',\n'
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>
' and\n'
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
';\nand they lived at the bottom of a well.'


In [41]:
for sibling in soup.find(id="link3").previous_siblings:
    print(repr(sibling))

' and\n'
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>
',\n'
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>
'Once upon a time there were three little sisters; and their names were\n'


### **Ida y vuelta**

Echa un vistazo al principio del documento de las "tres hermanas":
```
<html><head><title>The Dormouse's story</title></head>
<p class="title"><b>The Dormouse's story</b></p>
```

Un `HTML parser` toma este string de caracteres y lo convierte en una serie de eventos: "abrir una etiqueta `<html>`", "abrir una etiqueta `<head>`", "abrir una etiqueta `<title>`", "añadir un string", "cerrar la etiqueta `<title>`", "abrir una etiqueta `<p>`", etc. Beautiful Soup ofrece herramientas para reconstruir el análisis sintáctico inicial del documento.

#### **`.next_element` y `.previous_element`**

El atributo `.next_element` de un string o etiqueta apunta a lo que se haya parseado inmediatamente después. Puede ser el mismo que `.next_sibling`, pero suele ser drásticamente diferente.

Aquí está la última etiqueta `<a>` del documento "Tres hermanas". Su `.next_sibling` es un string: la conclusión de la frase que fue interrumpida por el inicio de la etiqueta `<a>`.:

In [42]:
last_a_tag = soup.find("a", id="link3")
last_a_tag

<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>

In [43]:
last_a_tag.next_sibling

';\nand they lived at the bottom of a well.'

Pero el  elemento `.next_element` de esa etiqueta `<a>`, lo que se parseo inmediatamente después de la etiqueta `<a>`, no es el resto de esa frase: es la palabra "Tillie":

In [44]:
last_a_tag.next_element

'Tillie'

Esto se debe a que en el marcado original, la palabra "Tillie" aparecía antes del punto y coma. El parser encontró una etiqueta `<a>`, luego la palabra "Tillie", luego la etiqueta de cierre `</a>`, luego el punto y coma y el resto de la frase. El punto y coma está en el mismo nivel que la etiqueta `<a>`, pero la palabra "Tillie" se encontró primero.

El atributo `.previous_element` es exactamente lo contrario de `.next_element`. Apunta al elemento que se parseó inmediatamente antes de éste:

In [45]:
last_a_tag.previous_element

' and\n'

In [46]:
last_a_tag.previous_element.next_element

<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>

#### **`.next_elements` y `.previous_elements`**

Ya deberías hacerte una idea. Puede utilizar estos iteradores para avanzar o retroceder en el documento a medida que se analiza:

In [47]:
for element in last_a_tag.next_elements:
    print(repr(element))

'Tillie'
';\nand they lived at the bottom of a well.'
'\n'
<p class="story">...</p>
'...'
'\n'


### **Búsqueda en el árbol**

Beautiful Soup define muchos métodos para buscar en el árbol de análisis sintáctico, pero todos son muy similares. Voy a pasar mucho tiempo explicando los dos métodos más populares: `find()` y `find_all()`. Los otros métodos toman casi exactamente los mismos argumentos, así que los cubriré brevemente.

Pasando un filtro a un argumento como `find_all()`, puede hacer zoom en las partes del documento que le interesen.

#### **`Tipos de filtros`**

Antes de hablar en detalle sobre `find_all()` y métodos similares, quiero mostrar ejemplos de diferentes filtros que se pueden pasar a estos métodos. Estos filtros aparecen una y otra vez en toda la API de búsqueda. Puedes usarlos para filtrar basándote en el nombre de una etiqueta, en sus atributos, en el texto de un string, o en alguna combinación de estos.

##### **Un string**

El filtro más simple es un string. Pase un string a un método de búsqueda y BeautifulSoup realizará una coincidencia con ese string exacto. Este código encuentra todas las etiquetas `<b>` del documento:

In [48]:
soup.find_all('b')

[<b>The Dormouse's story</b>]

Si pasas un string de bytes, Beautiful Soup asumirá que el string está codificado como UTF-8. Puedes evitar esto pasando un string Unicode en su lugar.

##### **Una expresión regular**

Si pasas un objeto de expresión regular, Beautiful Soup filtrará contra esa expresión regular usando su método `search()`. Este código encuentra todas las etiquetas cuyos nombres comienzan con la letra "`b`"; en este caso, la etiqueta `<body>` y la etiqueta `<b>`:

In [51]:
import re
for tag in soup.find_all(re.compile("^b")):
    print(tag.name)

body
b


Este código encuentra todas las etiquetas cuyos nombres contienen la letra "`t`":

In [52]:
for tag in soup.find_all(re.compile("t")):
    print(tag.name)

html
title


##### **Una lista**

Si pasas una lista, Beautiful Soup permitirá una coincidencia de string con cualquier elemento de esa lista. Este código encuentra todas las etiquetas `<a>` y todas las etiquetas `<b>`:

In [53]:
soup.find_all(["a", "b"])

[<b>The Dormouse's story</b>,
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

##### **`True`**

El valor `True` coincide con todo lo que puede. Este código encuentra todas las etiquetas del documento, pero ninguna de las cadenas de texto:

In [54]:
for tag in soup.find_all(True):
    print(tag.name)

html
head
title
body
p
b
p
a
a
a
p


##### **Una función**

Si ninguna de las otras coincidencias le sirve, defina una función que tome un elemento como único argumento. La función debe devolver `True` si el argumento coincide, y `False` en caso contrario.

Aquí tienes una función que devuelve `True` si una etiqueta define el atributo "`class`" pero no define el atributo "`id`":

In [57]:
def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')

Pasa esta función a `find_all()` y recogerás todas las etiquetas `<p>`:

In [64]:
soup.find_all(has_class_but_no_id)
# [<p class="title"><b>The Dormouse's story</b></p>,
#  <p class="story">Once upon a time there were...</p>,
#  <p class="story">...</p>]

[<p class="title"><b>The Dormouse's story</b></p>,
 <p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at the bottom of a well.</p>,
 <p class="story">...</p>]

Esta función sólo recoge las etiquetas `<p>`. No recoge las etiquetas `<a>`, porque esas etiquetas definen tanto "`class`" como "`id`". No recoge etiquetas como `<html>` y `<title>`, porque esas etiquetas no definen "class".

Si pasas una función para filtrar un atributo específico como `href`, el argumento pasado a la función será el valor del atributo, no toda la etiqueta. He aquí una función que encuentra todas las etiquetas cuyo atributo `href` no coincide con una expresión regular:

In [63]:
def not_lacie(href):
    return href and not re.compile("lacie").search(href)

soup.find_all(href=not_lacie)

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

La función puede ser tan complicada como necesites. He aquí una función que devuelve `True` si una etiqueta está rodeada de objetos de string:

In [66]:
from bs4 import NavigableString
def surrounded_by_strings(tag):
    return (isinstance(tag.next_element, NavigableString)
            and isinstance(tag.previous_element, NavigableString))

for tag in soup.find_all(surrounded_by_strings):
    print (tag.name)

body
p
a
a
a
p


#### **`find_all()`**

##### **Introducción**

> Sintaxis: **find_all(name, attrs, recursive, string, limit, `**kwargs`)**

El método `find_all()` busca entre los descendientes de una etiqueta y recupera todos los descendientes que coincidan con sus filtros. He dado varios ejemplos en Tipos de filtros, pero aquí hay algunos más:

In [67]:
soup.find_all("title")

[<title>The Dormouse's story</title>]

In [68]:
soup.find_all("p", "title")

[<p class="title"><b>The Dormouse's story</b></p>]

In [69]:
soup.find_all("a")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [70]:
soup.find_all(id="link2")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

In [73]:
soup.find_all(string="sisters")

[]

In [71]:
import re
soup.find(string=re.compile("sisters"))

'Once upon a time there were three little sisters; and their names were\n'

Algunas de ellas te resultarán familiares, pero otras son nuevas. ¿Qué significa pasar un valor como string, o id?. ¿Por qué `find_all("p", "title")` encuentra una etiqueta `<p>` con la clase CSS "title"? Veamos los argumentos de `find_all()`.

##### **El argumento `name`**

Introduce un valor para name y le dirás a BeautifulSoup que sólo considere etiquetas con ciertos nombres. Los strings de texto serán ignorados, al igual que las etiquetas cuyos nombres no coincidan.

Este es el uso más sencillo:

In [74]:
soup.find_all("title")

[<title>The Dormouse's story</title>]

Recuerde de Tipos de filtros que el valor a nombrar puede ser un string, una expresión regular, una lista, una función o el valor True.

##### **Los argumentos de las keywords**

Cualquier argumento que no sea reconocido se convertirá en un filtro sobre uno de los atributos de una etiqueta. Si pasas un valor para un argumento llamado `id`, BeautifulSoup filtrará contra el atributo '`id`' de cada etiqueta:

In [75]:
soup.find_all(id='link2')

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

Si pasa un valor para `href`, BeautifulSoup filtrará por el atributo '`href`' de cada etiqueta:

In [76]:
soup.find_all(href=re.compile("elsie"))

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

Puede filtrar un atributo basándose en un string, una expresión regular, una lista, una función o el valor True.

Este código encuentra todas las etiquetas cuyo atributo `id` tiene un valor, independientemente de cuál sea el valor:

In [77]:
soup.find_all(id=True)

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

Puede filtrar varios atributos a la vez introduciendo más de una keyword como argumento:

In [78]:
soup.find_all(href=re.compile("elsie"), id='link1')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

Algunos atributos, como los atributos `data-*` de HTML 5, tienen nombres que no pueden utilizarse como nombres de argumentos de keywords:

In [79]:
data_soup = BeautifulSoup('<div data-foo="value">foo!</div>')
data_soup.find_all(data-foo="value")

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (53685298.py, line 2)

Puede utilizar estos atributos en las búsquedas introduciéndolos en un diccionario y pasando el diccionario a `find_all()` como argumento `attrs`:

In [81]:
data_soup = BeautifulSoup('<div data-foo="value">foo!</div>')
data_soup.find_all(attrs={"data-foo": "value"})

[<div data-foo="value">foo!</div>]

No puedes utilizar un argumento de palabra clave para buscar el elemento '`name`' de HTML, porque `BeautifulSoup` utiliza el argumento `name` para contener el nombre de la propia etiqueta. En su lugar, puedes dar un valor a '`name`' en el argumento `attrs`:

In [82]:
name_soup = BeautifulSoup('<input name="email"/>')
name_soup.find_all(name="email")

[]

In [83]:
name_soup.find_all(attrs={"name": "email"})

[<input name="email"/>]

##### **Búsqueda por clase CSS**

Es muy útil buscar una etiqueta que tenga una determinada clase CSS, pero el nombre del atributo CSS, "`class`", es una palabra reservada en Python. Usar `class` como argumento de palabra clave te dará un error de sintaxis. A partir de Beautiful Soup 4.1.2, puedes buscar por CSS class usando el argumento de palabra clave `class_`:

In [84]:
soup.find_all("a", class_="sister")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

Como con cualquier argumento de keyword, puede pasar a `class_` un string, una expresión regular, una función o True:

In [85]:
soup.find_all(class_=re.compile("itl"))

[<p class="title"><b>The Dormouse's story</b></p>]

In [87]:
def has_six_characters(css_class):
    return css_class is not None and len(css_class) == 6

soup.find_all(class_=has_six_characters)

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

Recuerda que una misma etiqueta puede tener varios valores para su atributo "`class`". Al buscar una etiqueta que coincida con una determinada CSS class, se está comparando con cualquiera de sus CSS classes:

In [88]:
css_soup = BeautifulSoup('<p class="body strikeout"></p>')
css_soup.find_all("p", class_="strikeout")

[<p class="body strikeout"></p>]

In [89]:
css_soup.find_all("p", class_="body")

[<p class="body strikeout"></p>]

También puede buscar el valor exacto del string del atributo `class`:

In [90]:
css_soup.find_all("p", class_="body strikeout")

[<p class="body strikeout"></p>]

Pero la búsqueda de variantes del valor del string no funcionará:

In [91]:
css_soup.find_all("p", class_="strikeout body")

[]

Si desea buscar etiquetas que coincidan con dos o más CSS classes, debe utilizar un selector CSS:

In [92]:
css_soup.select("p.strikeout.body")

[<p class="body strikeout"></p>]

En versiones anteriores de Beautiful Soup, que no tienen el atajo `class_`, puedes usar el truco de attrs mencionado anteriormente. Crea un diccionario cuyo valor para "`class`" sea el string (o expresión regular, o lo que sea) que quieras buscar:

In [93]:
soup.find_all("a", attrs={"class": "sister"})

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

##### **El argumento `string`**   

Con string puede buscar strings en lugar de etiquetas. Al igual que con name y los argumentos de keywords, puede pasar un string, una expresión regular, una lista, una función o el valor True. He aquí algunos ejemplos:

In [94]:
soup.find_all(string="Elsie")

['Elsie']

In [95]:
soup.find_all(string=["Tillie", "Elsie", "Lacie"])

['Elsie', 'Lacie', 'Tillie']

In [96]:
soup.find_all(string=re.compile("Dormouse"))

["The Dormouse's story", "The Dormouse's story"]

In [98]:
def is_the_only_string_within_a_tag(s):
    """Devuelve True si este string es el único hijo de su etiqueta padre."""
    return (s == s.parent.string)

soup.find_all(string=is_the_only_string_within_a_tag)

["The Dormouse's story",
 "The Dormouse's story",
 'Elsie',
 'Lacie',
 'Tillie',
 '...']

Aunque string es para encontrar strings, puedes combinarlo con argumentos que encuentren tags: BeautifulSoup encontrará todas las etiquetas cuyo `.string` coincida con tu valor para string. Este código encuentra las etiquetas `<a>` cuyo `.string` es "`Elsie`":

In [101]:
soup.find_all("a", string="Elsie")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

El argumento `string` es nuevo en Beautiful Soup 4.4.0. En versiones anteriores se llamaba `text`:

In [102]:
soup.find_all("a", text="Elsie")

C:\Users\Alfonso\AppData\Local\Temp\ipykernel_28828\851768435.py:1: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  soup.find_all("a", text="Elsie")


[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

##### **El argumento `limit`**  

`find_all()` devuelve todas las etiquetas y strings que coinciden con sus filtros. Esto puede tardar un poco si el documento es grande. Si no necesita todos los resultados, puede pasar un número como límite. Esto funciona igual que la palabra clave LIMIT en SQL. Le dice a BeautifulSoup que deje de recoger resultados después de haber encontrado un cierto número.

Hay tres enlaces en el documento "tres hermanas", pero este código sólo encuentra los dos primeros:

In [103]:
soup.find_all("a", limit=2)

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

##### **El argumento `recursive`**  

Si llamas a `mytag.find_all()`, Beautiful Soup examinará todos los descendientes de `mytag`: sus hijos, los hijos de sus hijos, y así sucesivamente. Si sólo quieres que BeautifulSoup considere los hijos directos, puedes pasar `recursive = False`. Vea la diferencia aquí:

In [104]:
soup.html.find_all("title")

[<title>The Dormouse's story</title>]

In [105]:
soup.html.find_all("title", recursive=False)

[]

Aquí está esa parte del documento:
```
<html>
 <head>
  <title>
   The Dormouse's story
  </title>
 </head>
...
```

#### **`Llamar a una etiqueta es como llamar a find_all()`**

##### **Introducción**

Como `find_all()` es el método más popular de la API de búsqueda de Beautiful Soup, puedes usar un atajo para él. Si tratas el objeto BeautifulSoup o un objeto Tag como si fuera una función, entonces es lo mismo que llamar a `find_all()` en ese objeto. Estas dos líneas de código son equivalentes:

In [3]:
soup.find_all("a")
soup("a")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

Estas dos líneas también son equivalentes:

In [5]:
soup.title.find_all(string=True)

["The Dormouse's story"]

In [6]:
soup.title(string=True)

["The Dormouse's story"]

##### **`find()`**

> Sintaxis: **find(name, attrs, recursive, string, `**kwargs`)**

El método `find_all()` explora todo el documento en busca de resultados, pero a veces sólo se desea encontrar un resultado. Si sabes que un documento sólo tiene una etiqueta `<body>`, es una pérdida de tiempo escanear todo el documento buscando más. En lugar de pasar `limit = 1` cada vez que llame a find_all, puede utilizar el método `find()`. Estas dos líneas de código son casi equivalentes:

In [7]:
soup.find_all('title', limit=1)

[<title>The Dormouse's story</title>]

In [8]:
soup.find('title')

<title>The Dormouse's story</title>

La única diferencia es que `find_all()` devuelve una lista que contiene el resultado único, y `find()` sólo devuelve el resultado.

Si `find_all()` no encuentra nada, devuelve una lista vacía. Si `find()` no encuentra nada, devuelve `None`:

In [9]:
print(soup.find("nosuchtag"))

None


¿Recuerdas el truco `soup.head.title` de Navegar usando nombres de etiquetas? Ese truco funciona llamando repetidamente a `find()`:

In [11]:
soup.head

<head><title>The Dormouse's story</title></head>

In [10]:
soup.head.title

<title>The Dormouse's story</title>

In [12]:
soup.find("head").find("title")

<title>The Dormouse's story</title>

##### **`find_parents()` y `find_parent()`**

> Sintaxis: **find_parents(name, attrs, string, limit, `**kwargs`)**

> Sintaxis: **find_parent(name, attrs, string, `**kwargs`)**

He dedicado mucho tiempo a `find_all()` y `find()`. La API de BeautifulSoup define otros diez métodos para buscar en el árbol, pero no te asustes. Cinco de estos métodos son básicamente los mismos que `find_all()`, y los otros cinco son básicamente los mismos que `find()`. Las únicas diferencias están en qué partes del árbol buscan.

Primero consideremos `find_parents()` y `find_parent()`. Recuerde que `find_all()` y `find()` trabajan hacia abajo en el árbol, buscando en los descendientes de la etiqueta. Estos métodos hacen lo contrario: trabajan hacia arriba en el árbol, buscando los padres de una etiqueta (o string). Vamos a probarlos, partiendo de un string enterrado en las profundidades del documento "tres hijas":

In [13]:
a_string = soup.find(string="Lacie")
a_string

'Lacie'

In [14]:
a_string.find_parents("a")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

In [15]:
a_string.find_parent("p")

<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

In [17]:
a_string.find_parents("p", class_="title")

[]

Una de las tres etiquetas `<a>` es el padre directo del string en cuestión, por lo que nuestra búsqueda lo encuentra. Una de las tres etiquetas `<p>` es un padre indirecto del string, y nuestra búsqueda también lo encuentra. Hay una etiqueta `<p>` con la clase CSS "`title`" en algún lugar del documento, pero no es uno de los padres de este string, por lo que no podemos encontrarla con `find_parents()`.

Es posible que haya hecho la conexión entre `find_parent()` y `find_parents()`, y los atributos `.parent` y `.parents` mencionados anteriormente. La conexión es muy fuerte. Estos métodos de búsqueda utilizan .parents para iterar sobre todos los padres, y comprobar cada uno contra el filtro proporcionado para ver si coincide.

##### **`find_next_siblings()` y `find_next_sibling()`**

> Sintaxis: **find_next_siblings(name, attrs, string, limit, `**kwargs`)**

> Sintaxis: **find_next_sibling(name, attrs, string, `**kwargs`)**

Estos métodos utilizan `.next_siblings` para iterar sobre el resto de los hermanos de un elemento en el árbol. El método `find_next_siblings()` devuelve todos los hermanos que coinciden, y `find_next_sibling()` sólo devuelve el primero:

In [18]:
first_link = soup.a
first_link

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [19]:
first_link.find_next_siblings("a")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [24]:
first_story_paragraph = soup.find("p", "story")
first_story_paragraph

<p class="story">Once upon a time there were three little sisters; and their names were
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

In [25]:
first_story_paragraph.find_previous_sibling("p")

<p class="title"><b>The Dormouse's story</b></p>

##### **`find_all_next()` y `find_next()`**

> Sintaxis: **find_all_next(name, attrs, string, limit, `**kwargs`)**

> Sintaxis: **find_next(name, attrs, string, `**kwargs`)**

Estos métodos utilizan `.next_elements` para iterar sobre las etiquetas y strings que vienen a continuación en el documento. El método `find_all_next()` devuelve todas las coincidencias, y `find_next()` sólo devuelve la primera coincidencia:

In [26]:
first_link = soup.a
first_link

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [27]:
first_link.find_all_next(string=True)

['Elsie',
 ',\n',
 'Lacie',
 ' and\n',
 'Tillie',
 ';\nand they lived at the bottom of a well.',
 '\n',
 '...',
 '\n']

In [28]:
first_link.find_next("p")

<p class="story">...</p>

En el primer ejemplo, aparecía la cadena "`Elsie`", aunque estaba contenida dentro de la etiqueta `<a>` de la que partimos. En el segundo ejemplo, aparece la última etiqueta `<p>` del documento, aunque no esté en la misma parte del árbol que la etiqueta `<a>` de la que partimos. Para estos métodos, lo único que importa es que un elemento coincida con el filtro y aparezca más adelante en el documento que el elemento inicial.

##### **`find_all_previous()` y `find_previous()`**

> Sintaxis: **find_all_previous(name, attrs, string, limit, `**kwargs`)**

> Sintaxis: **find_previous(name, attrs, string, `**kwargs`)**

Estos métodos utilizan `.previous_elements` para recorrer las etiquetas y strings anteriores en el documento. El método `find_all_previous()` devuelve todas las coincidencias, y `find_previous()` sólo devuelve la primera coincidencia:

In [29]:
first_link = soup.a
first_link

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

In [30]:
first_link.find_all_previous("p")

[<p class="story">Once upon a time there were three little sisters; and their names were
 <a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a> and
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>;
 and they lived at the bottom of a well.</p>,
 <p class="title"><b>The Dormouse's story</b></p>]

In [31]:
first_link.find_previous("title")


<title>The Dormouse's story</title>

La llamada a `find_all_previous("p")` encontró el primer párrafo del documento (el que tiene `class="title"`), pero también encuentra el segundo párrafo, la etiqueta `<p>` que contiene la etiqueta `<a>` con la que empezamos. Esto no debería ser demasiado sorprendente: estamos viendo todas las etiquetas que aparecen antes en el documento que aquella con la que comenzamos. Una etiqueta `<p>` que contiene una etiqueta `<a>` debe haber aparecido antes de la etiqueta `<a>` que contiene.

#### **`CSS Selectors`**

##### **`select()`**

A partir de la versión 4.7.0, BeautifulSoup soporta la mayoría de los selectores CSS4 a través del proyecto SoupSieve. Si instalaste Beautiful Soup a través de `pip`, SoupSieve fue instalado al mismo tiempo, así que no tienes que hacer nada extra.

BeautifulSoup tiene un método `.select()` que utiliza SoupSieve para ejecutar un selector CSS contra un documento parseado y devolver todos los elementos coincidentes. Tag tiene un método similar que ejecuta un selector CSS contra el contenido de una sola etiqueta.

(Las versiones anteriores de Beautiful Soup también tienen el método `.select()`, pero sólo se soportan los selectores CSS más utilizados).

La documentación de SoupSieve lista todos los selectores CSS actualmente soportados, pero aquí están algunos de los básicos:

Puedes encontrar etiquetas:

In [32]:
soup.select("title")

[<title>The Dormouse's story</title>]

In [33]:
soup.select("p:nth-of-type(3)")

[<p class="story">...</p>]

Buscar etiquetas debajo de otras etiquetas

In [34]:
soup.select("body a")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [35]:
soup.select("html head title")

[<title>The Dormouse's story</title>]

Encuentra etiquetas directamente debajo de otras etiquetas:

In [36]:
soup.select("head > title")

[<title>The Dormouse's story</title>]

In [37]:
soup.select("p > a")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [38]:
soup.select("p > a:nth-of-type(2)")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

In [39]:
soup.select("p > #link1")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

In [40]:
soup.select("body > a")

[]

Encuentra a los hermanos de las etiquetas:

In [41]:
soup.select("#link1 ~ .sister")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [42]:
soup.select("#link1 + .sister")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

Buscar etiquetas por clase CSS:

In [43]:
soup.select(".sister")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [44]:
soup.select("[class~=sister]")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

Buscar etiquetas por ID:

In [45]:
soup.select("#link1")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

In [46]:
soup.select("a#link2")

[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

Buscar etiquetas que coincidan con cualquier selector de una lista de selectores:

In [47]:
soup.select("#link1,#link2")

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]

Comprueba la existencia de un atributo:

In [48]:
soup.select('a[href]')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

Buscar etiquetas por valor de atributo:

In [49]:
soup.select('a[href="http://example.com/elsie"]')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

In [50]:
soup.select('a[href^="http://example.com/"]')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [51]:
soup.select('a[href$="tillie"]')

[<a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [52]:
soup.select('a[href*=".com/el"]')

[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>]

##### **`select_one()`**

También hay un método llamado `select_one()`, que encuentra sólo la primera etiqueta que coincide con un selector:

In [ ]:
soup.select_one(".sister")

### **Modificar el árbol**

#### **Introducción**

Ya he hablado de esto antes, en Atributos, pero merece la pena repetirlo. Puedes renombrar una etiqueta, cambiar los valores de sus atributos, añadir nuevos atributos y borrar atributos:

In [6]:
soup = BeautifulSoup('<b class="boldest">Extremely bold</b>')
tag = soup.b

In [7]:
tag.name = "blockquote"
tag['class'] = 'verybold'
tag['id'] = 1
tag

<blockquote class="verybold" id="1">Extremely bold</blockquote>

In [8]:
del tag['class']
del tag['id']
tag

<blockquote>Extremely bold</blockquote>

#### **Modificación de `.string`**

Si asignas al atributo `.string` de una etiqueta un nuevo valor string, el contenido de la etiqueta se sustituye por ese valor string:

In [9]:
markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)

tag = soup.a
tag.string = "New link text."
tag

<a href="http://example.com/">New link text.</a>

Cuidado: si la etiqueta contenía otras etiquetas, éstas y todo su contenido serán destruidos.

##### **`append()`**

Puedes añadir contenido a una etiqueta con `Tag.append()`. Funciona igual que llamar a `.append()` en una lista de Python:

In [10]:
soup = BeautifulSoup("<a>Foo</a>")
soup.a.append("Bar")

soup

<html><body><a>FooBar</a></body></html>

In [11]:
soup.a.contents

['Foo', 'Bar']

##### **`NavigableString()` y `.new_tag()`**

Si necesitas añadir un string a un documento, no hay problema-puedes pasar un string de Python a `append()`, o puedes llamar al constructor `NavigableString`:

In [21]:
from bs4 import NavigableString

soup = BeautifulSoup("<b></b>")
tag = soup.b
tag.append("Hello")
tag

<b>Hello</b>

In [22]:
new_string = NavigableString(" there")
tag.append(new_string)
tag

<b>Hello there</b>

In [23]:
tag.contents

['Hello', ' there']

Si quieres crear un comentario o alguna otra subclase de `NavigableString`, simplemente llama al constructor:

In [24]:
from bs4 import Comment
new_comment = Comment("Nice to see you.")
tag.append(new_comment)
tag

<b>Hello there<!--Nice to see you.--></b>

In [25]:
tag.contents

['Hello', ' there', 'Nice to see you.']

(Esta es una nueva función de Beautiful Soup 4.4.0.)

¿Qué pasa si necesitas crear una etiqueta completamente nueva? La mejor solución es llamar al método de fábrica `BeautifulSoup.new_tag()`:

In [26]:
soup = BeautifulSoup("<b></b>")
original_tag = soup.b

new_tag = soup.new_tag("a", href="http://www.example.com")
original_tag.append(new_tag)
original_tag

<b><a href="http://www.example.com"></a></b>

In [27]:
new_tag.string = "Link text."
original_tag

<b><a href="http://www.example.com">Link text.</a></b>

Sólo se requiere el primer argumento, el nombre de la etiqueta.

`Tag.insert()` es igual que `Tag.append()`, excepto que el nuevo elemento no va necesariamente al final del `.contents` de su padre. Se insertará en la posición numérica que tú digas. Funciona igual que `.insert()` en una lista Python:

`Tag.insert()` es igual que `Tag.append()`, excepto que el nuevo elemento no va necesariamente al final del `.contents` de su padre. Se insertará en la posición numérica que tú digas. Funciona igual que `.insert()` en una lista Python:

In [ ]:
from bs4 import BeautifulSoup
markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)
tag = soup.a
tag

<a href="http://example.com/">I linked to <i>example.com</i></a>

In [ ]:
tag.insert(1, "but did not endorse ")
tag

<a href="http://example.com/">I linked to but did not endorse <i>example.com</i></a>

In [ ]:
tag.contents

['I linked to ', 'but did not endorse ', <i>example.com</i>]

##### **`insert()`**

`Tag.insert()` es igual que `Tag.append()`, excepto que el nuevo elemento no va necesariamente al final del `.contents` de su padre. Se insertará en la posición numérica que tú digas. Funciona igual que `.insert()` en una lista Python:

In [ ]:
from bs4 import BeautifulSoup
markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)
tag = soup.a
tag

<a href="http://example.com/">I linked to <i>example.com</i></a>

In [ ]:
tag.insert(1, "but did not endorse ")
tag

<a href="http://example.com/">I linked to but did not endorse <i>example.com</i></a>

In [ ]:
tag.contents

['I linked to ', 'but did not endorse ', <i>example.com</i>]

##### **`insert_before()` e `insert_after()`**

El método `insert_before()` inserta etiquetas o strings inmediatamente antes de otra cosa en el árbol de análisis sintáctico:

In [ ]:
soup = BeautifulSoup("<b>stop</b>")
tag = soup.new_tag("i")
tag.string = "Don't"
soup.b.string.insert_before(tag)
soup.b

<b><i>Don't</i>stop</b>

El método `insert_after()` inserta etiquetas o strings inmediatamente después de otra cosa en el árbol de análisis sintáctico:

In [ ]:
div = soup.new_tag('div')
div.string = 'ever'
soup.b.i.insert_after(" you ", div)
soup.b

<b><i>Don't</i> you <div>ever</div>stop</b>

In [ ]:
soup.b.contents

[<i>Don't</i>, ' you ', <div>ever</div>, 'stop']

##### **`clear()`**

`Tag.clear()` elimina el contenido de una etiqueta:

In [ ]:
markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)
tag = soup.a

tag.clear()
tag

<a href="http://example.com/"></a>

##### **`extraer()`**

`PageElement.extract()` extrae una etiqueta o string del árbol. Devuelve la etiqueta o string que se extrajo:

In [ ]:
markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)
a_tag = soup.a

In [ ]:
i_tag = soup.i.extract()

a_tag

<a href="http://example.com/">I linked to </a>

In [ ]:
i_tag

<i>example.com</i>

In [ ]:
print(i_tag.parent)

None


En este punto tienes dos árboles de análisis: uno enraizado en el objeto BeautifulSoup que usaste para analizar el documento, y otro enraizado en la etiqueta que fue extraída. Puedes continuar llamando a extract en un hijo del elemento que extrajiste:

In [ ]:
my_string = i_tag.string.extract()
my_string

'example.com'

In [ ]:
print(my_string.parent)

None


In [ ]:
i_tag

<i></i>

##### **`decompose()`**

`Tag.decompose()` elimina una etiqueta del árbol y, a continuación, la destruye completamente junto con su contenido:

In [ ]:
from bs4 import BeautifulSoup

markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)
a_tag = soup.a

soup.i.decompose()

a_tag

<a href="http://example.com/">I linked to </a>

##### **`replace_with()`**

`PageElement.replace_with()` elimina una etiqueta o string del árbol y la sustituye por la etiqueta o string de su elección:

In [ ]:
markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)
a_tag = soup.a

new_tag = soup.new_tag("b")
new_tag.string = "example.net"
a_tag.i.replace_with(new_tag)

a_tag

<a href="http://example.com/">I linked to <b>example.net</b></a>

`replace_with()` devuelve la etiqueta o string reemplazada, para que pueda examinarla o añadirla de nuevo a otra parte del árbol.

##### **`wrap()`**

`PageElement.wrap()` envuelve un elemento en la etiqueta especificada. Devuelve la nueva envoltura:

In [ ]:
soup = BeautifulSoup("<p>I wish I was bold.</p>")
soup.p.string.wrap(soup.new_tag("b"))

<b>I wish I was bold.</b>

In [ ]:
soup.p.wrap(soup.new_tag("div"))

<div><p><b>I wish I was bold.</b></p></div>

##### **`unwrap()`**

`Tag.unwrap()` es lo contrario de wrap(). Reemplaza una etiqueta con lo que hay dentro de esa etiqueta. Sirve para eliminar marcas:

In [ ]:
markup = '<a href="http://example.com/">I linked to <i>example.com</i></a>'
soup = BeautifulSoup(markup)
a_tag = soup.a

a_tag.i.unwrap()
a_tag

<a href="http://example.com/">I linked to example.com</a>

Al igual que `replace_with()`, `unwrap()` devuelve la etiqueta reemplazada.

##### **`smooth()`**

Después de llamar a un montón de métodos que modifican el árbol de análisis, puedes acabar con dos o más objetos NavigableString uno al lado del otro. Beautiful Soup no tiene ningún problema con esto, pero como no puede ocurrir en un documento recién parseado, puede que no esperes un comportamiento como el siguiente:

In [ ]:
soup = BeautifulSoup("<p>A one</p>")
soup.p.append(", a two")

soup.p.contents

['A one', ', a two']

In [ ]:
print(soup.p.encode())

b'<p>A one, a two</p>'


In [ ]:
print(soup.p.prettify())

<p>
 A one
 , a two
</p>



Puede llamar a `Tag.smooth()` para limpiar el árbol de análisis consolidando strings adyacentes:

In [ ]:
soup.smooth()

soup.p.contents

['A one, a two']

In [ ]:
print(soup.p.prettify())

<p>
 A one, a two
</p>

